# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import pprint
from scipy.stats import linregress
import scipy.stats as st
import datetime as dt     

# Import API key
from api_keys import weather_api_key2

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

ModuleNotFoundError: No module named 'api_keys'

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).


In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?units=imperial"

query_url = f"{url}&appid={weather_api_key2}&q="

city_name = []
lat = []
lng = []
max_temp = []
hum = []
cloud = []
wind_spd = []
country = []
date = []

#create counter
city_count = 1
set_count = 1

for i, city in enumerate(cities):
    if (i %50 == 0 and i >= 50):
        set_count = set_count + 1
        city_count = 0
        city_count +=1
    #city_url = query_url + "&q=" + city
    print("Processing record %s of Set %s | %s" % (city_count, set_count, city))
    
#   parameters = {"q":city,
#                "units":"imperial",
#              "appid": weather_api_key2}
    try:
        city_data = requests.get(query_url+city)
        response_json = city_data.json()
        city_name.append(response_json["name"])
        lat.append(response_json["coord"]["lat"])
        lng.append(response_json["coord"]["lon"])
        max_temp.append(response_json["main"]["temp_max"])
        hum.append(response_json["main"]["humidity"])
        cloud.append(response_json["clouds"]["all"])
        wind_spd.append(response_json["wind"]["speed"])
        country.append(response_json["sys"]["country"])
        date.append(response_json["dt"])

    except:
        print("City not found. Skipping...")

print("-------Complete--------")

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# create a data frame from cities, lat, and temp
weather_data = pd.DataFrame({"City": city_name,
                             "Latitude": lat,
                             "Longitude": lng,
                             "Max Temp": max_temp,
                             "Humidity": hum,
                             "Cloudiness": cloud,
                             "Wind Speed": wind_spd,
                             "Country": country,
                             "Date": date})
weather_data.head()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
humid_cities = weather_data.loc[weather_data['Humidity'] > 100]
humid_cities

In [ ]:
# Export the City_Data into a csv
weather_data.to_csv('../output_data/output_cities_AR.csv',sep=',', header=True, index=False)

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Create a Scatter Plot for lat v temp
x_temp = weather_data['Max Temp']
y_temp = weather_data['Latitude']
plt.scatter(x_temp, y_temp, c= 'yellow', edgecolor='orange')
plt.xlabel('Temperature')
plt.ylabel('Latitude')
date = dt.date.today()
plt.title("City Latitude v. Max Temperature   " + str(date))
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Create a Scatter Plot for lat v humidity
x_hum = weather_data['Humidity']
y_hum = weather_data['Latitude']
plt.scatter(x_hum,y_hum, color='lightblue', edgecolors='b')
plt.xlabel('Humidity')
plt.ylabel('Latitude')
plt.title("City Latitude v. Humidity   " + str(date))
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Create a Scatter Plot for lat v cloudiness
x_cloud = weather_data['Cloudiness']
y_cloud = weather_data['Latitude']
plt.scatter(x_cloud, y_cloud, color='lightgrey', edgecolors='black')
plt.xlabel('Cloudiness')
plt.ylabel('Latitude')
plt.title("City Latitude v. Cloudiness   " + str(date))
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# Create a Scatter Plot for lat v wind speed
x_wind = weather_data['Wind Speed']
y_wind = weather_data['Latitude']
plt.scatter(x_wind,y_wind, c = 'white', edgecolor = 'gray')
plt.xlabel('Wind Speed')
plt.ylabel('Latitude')
plt.title("City Latitude v. Wind Speed   " + str(date))
plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
no_hem = weather_data.loc[weather_data['Latitude'] > 0]

In [ ]:
so_hem = weather_data.loc[weather_data['Latitude'] < 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
n_x_temp = no_hem['Max Temp']
n_y_temp = no_hem['Latitude']

(slope, intercept, rvalue, pvalue, stderr) = linregress(n_x_temp, n_y_temp)
regress_values = n_x_temp * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(f"The r-squared is: {rvalue**2}")
plt.scatter(n_x_temp, n_y_temp, c = 'yellow', edgecolor = 'orange')
plt.plot(n_x_temp,regress_values,"r-")
plt.annotate(line_eq, (10,10), fontsize = 15, color = 'black')
plt.xlabel('Temperature')
plt.ylabel('Latitude')
plt.title("N. Hemisphere City Latitude v. Temperature   " + str(date))

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
s_x_temp = so_hem['Max Temp']
s_y_temp = so_hem['Latitude']

(slope, intercept, rvalue, pvalue, stderr) = linregress(s_x_temp, s_y_temp)
regress_values = s_x_temp * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(f"The r-squared is: {rvalue**2}")
plt.scatter(s_x_temp, s_y_temp, c = 'yellow', edgecolor = 'red')
plt.plot(s_x_temp,regress_values,"r-")
plt.annotate(line_eq, (70,-50), fontsize = 15, color='black')
plt.xlabel('Temperature')
plt.ylabel('Latitude')
plt.title("S. Hemisphere City Latitude v. Temperature   " + str(date))

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
n_x_hum = no_hem['Humidity']
n_y_hum = no_hem['Latitude']

(slope, intercept, rvalue, pvalue, stderr) = linregress(n_x_hum, n_y_hum)
regress_values = n_x_hum * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(f"The r-squared is: {rvalue**2}")
plt.scatter(n_x_hum,n_y_hum, color='lightblue', edgecolors='b')
plt.plot(n_x_hum,regress_values,"r-")
plt.annotate(line_eq, (5,72), fontsize = 15, color = 'black')
plt.xlabel('Humidity')
plt.ylabel('Latitude')
plt.title("N. Hemisphere City Latitude v. Humidity   " + str(date))

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
s_x_hum = so_hem['Humidity']
s_y_hum = so_hem['Latitude']

(slope, intercept, rvalue, pvalue, stderr) = linregress(s_x_hum, s_y_hum)
regress_values = s_x_hum * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(f"The r-squared is: {rvalue**2}")
plt.scatter(s_x_hum,s_y_hum, color='lightblue', edgecolors='darkblue')
plt.plot(s_x_hum,regress_values,"r-")
plt.annotate(line_eq, (10,-55), fontsize = 15, color = 'black')
plt.xlabel('Humidity')
plt.ylabel('Latitude')
plt.title("S. Hemisphere City Latitude v. Humidity   " + str(date))

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
n_x_cloud = no_hem['Cloudiness']
n_y_cloud = no_hem['Latitude']

(slope, intercept, rvalue, pvalue, stderr) = linregress(n_x_cloud, n_y_cloud)
regress_values = n_x_cloud * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(f"The r-squared is: {rvalue**2}")
plt.scatter(n_x_cloud,n_y_cloud, color='lightgreen', edgecolors='black')
plt.plot(n_x_cloud,regress_values,"r-")
plt.annotate(line_eq, (30,75), fontsize = 15, color = 'black')
plt.xlabel('Cloudiness')
plt.ylabel('Latitude')
plt.title("N. Hemisphere City Latitude v. Cloudiness   " + str(date))

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
s_x_cloud = so_hem['Cloudiness']
s_y_cloud = so_hem['Latitude']

(slope, intercept, rvalue, pvalue, stderr) = linregress(s_x_cloud, s_y_cloud)
regress_values = s_x_cloud * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(f"The r-squared is: {rvalue**2}")
plt.scatter(s_x_cloud, s_y_cloud, color='black', edgecolors='darkgreen')
plt.plot(s_x_cloud,regress_values,"r-")
plt.annotate(line_eq, (0,-55), fontsize = 15, color = 'black')
plt.xlabel('Cloudiness')
plt.ylabel('Latitude')
plt.title("S. Hemisphere City Latitude v. Cloudiness   " + str(date))

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
n_x_wind = no_hem['Wind Speed']
n_y_wind = no_hem['Latitude']

(slope, intercept, rvalue, pvalue, stderr) = linregress(n_x_wind, n_y_wind)
regress_values = n_x_wind * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(f"The r-squared is: {rvalue**2}")
plt.scatter(n_x_wind, n_y_wind, color='purple', edgecolors='gray')
plt.plot(n_x_wind,regress_values,"r-")
plt.annotate(line_eq, (14,75), fontsize = 15, color = 'black')
plt.xlabel('Wind Speed (mph)')
plt.ylabel('Latitude')
plt.title("N. Hemisphere City Latitude v. Wind Speed (mph)   " + str(date))

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
s_x_wind = so_hem['Wind Speed']
s_y_wind = so_hem['Latitude']

(slope, intercept, rvalue, pvalue, stderr) = linregress(s_x_wind, s_y_wind)
regress_values = s_x_wind * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(f"The r-squared is: {rvalue**2}")
plt.scatter(s_x_wind, s_y_wind, color='gray', edgecolors='purple')
plt.plot(s_x_wind,regress_values,"r-")
plt.annotate(line_eq, (15, -32), fontsize = 15, color = 'black')
plt.xlabel('Wind Speed (mph)')
plt.ylabel('Latitude')
plt.title("S. Hemisphere City Latitude v. Wind Speed (mph)   " + str(date))